In [26]:
from itertools import combinations

import pandas as pd
from shapely import intersection
from geopandas import GeoDataFrame, sjoin
from colocationpatterns.sample_data import generate_sample_data

In [41]:
class ColocationMiner:

    def __init__(
        self,
        data: GeoDataFrame,
        feature_type_column: str,
        feature_type_unique_id_column: str,
        neighbourhood: float):

        self.data = data # all events instances (id, event tpy, location, other attributes)
        self.feature_type_column = feature_type_column # column to recognize event type
        self.feature_type_unique_id_column = feature_type_unique_id_column # column with id unique within event type
        self.ET = set(data[feature_type_column].unique()) # set of event types
        self.R = neighbourhood # max distance to consider neighbour as colocation
        self.K = data[feature_type_column].nunique() # number of event types
        self.tables = {k: {} for k in range(1, self.K+1)} # structure to store tables based on size-k colocation
        self.statistics = None
        self.colocations = None

    def calculate_participation_ratio(self, candidate_table, feature_type):

        all_feature_types = len(self.tables[1][feature_type])
        features_in_colocation = candidate_table[feature_type].nunique()

        return features_in_colocation/all_feature_types

    def calculate_participation_index(self, participation_ratios):

        return min(participation_ratios)

    def calculate_conditional_probalility(self):

        pass

    def merge_by_neighbourhood(self, tables_id:tuple):

        result = self.tables[1][tables_id[0]].copy()
        result['geometry'] = result['geometry'].buffer(self.R)
        
        for table_id in tables_id[1::]:
        
            table = self.tables[1][table_id].copy()
            merged = sjoin(result, table)
            if merged.empty: return None
            merged.drop(columns='index_right', inplace=True)
            merged = pd.merge(merged, table, left_on=table_id, right_on=table_id)
            merged['geometry_y'] = merged['geometry_y'].buffer(self.R)
            merged['geometry'] = merged.apply(lambda row: intersection(row['geometry_x'], row['geometry_y']), axis=1)
            result = merged.drop(columns=['geometry_x', 'geometry_y'])
    
        return result
    
    
    def mine(self, min_participation_index: float, store_higher_lvl_tables:bool=False, verbose:bool=False):

        statistics = {}

        # Generate co-location candidates and compute statistics
        for k in range(1, self.K+1):
            
            for colocation in combinations(self.ET, k):

                if k == 1: # elementary tables
                    if verbose: print('Creating elementary tables')
                    self.tables[k] =  { table_id:
                        GeoDataFrame(
                            [{
                                table_id: getattr(event, self.feature_type_unique_id_column),
                                'geometry': event.geometry
                            } for event in table[[self.feature_type_unique_id_column, 'geometry']].itertuples()]
                        ) for table_id, table in self.data.groupby(by=self.feature_type_column, as_index=False)
                    }
                    break

                if verbose: print(f'Creating table k_level = {k} for tables {colocation}')
                table = self.merge_by_neighbourhood(colocation)
                if table is None:
                    if verbose: print(f'No points for co-location candidate {colocation} to match neighbourhood definition R={self.R}')
                    continue
                if store_higher_lvl_tables:
                    self.tables[k][colocation] = table
                
                statistics[colocation] = {'participation_ratios': {}}
                for feature_type in colocation:
                    pr = self.calculate_participation_ratio(table, feature_type)
                    statistics[colocation]['participation_ratios'][feature_type] = pr
                    if verbose: print(f'\tParticipation ratio for {feature_type} in colocation {colocation} is {pr}')
                
                pi = self.calculate_participation_index(statistics[colocation]['participation_ratios'].values())
                statistics[colocation]['participation_index'] = pi
                if verbose: print(f'\tParticipation index for colocation {colocation} is {pi}')

                statistics[colocation]['colocation'] = pi >= min_participation_index
    
        self.statistics = statistics
        self.colocations = list(filter(lambda key: self.statistics[key]['colocation'], self.statistics.keys()))

        return self.colocations

In [42]:
data = generate_sample_data()

In [43]:
cm = ColocationMiner(data, 'spatial_feature_type', 'instance_id', 2.95)

In [44]:
cm.mine(0.5, store_higher_lvl_tables=True)

[('B', 'C'), ('C', 'A')]

In [45]:
cm.tables

{1: {'A':    A           geometry
  0  1  POINT (1.75 1.75)
  1  2  POINT (5.45 3.75)
  2  3   POINT (8.5 3.25)
  3  4     POINT (4.25 5),
  'B':    B            geometry
  0  1     POINT (2.1 0.5)
  1  2  POINT (10.65 0.85)
  2  3   POINT (13.74 3.9)
  3  4   POINT (6.25 1.35)
  4  5      POINT (15.1 6),
  'C':    C          geometry
  0  1    POINT (8.65 1)
  1  2   POINT (0.1 3.3)
  2  3  POINT (14.2 5.5)},
 2: {('B',
   'C'):    B  C                                           geometry
  0  2  1  POLYGON ((9.25938 -1.75167, 9.01107 -1.60284, ...
  1  3  3  POLYGON ((11.28716 5.53893, 11.45962 5.77146, ...
  2  4  1  POLYGON ((9.18579 1.06085, 9.14332 0.77448, 9....
  3  5  3  POLYGON ((15.95634 3.17703, 15.67552 3.10668, ...,
  ('B',
   'A'):    B  A                                           geometry
  0  1  1  POLYGON ((-0.85 0.5, -0.83579 0.78915, -0.7933...
  1  4  3  POLYGON ((9.18579 1.06085, 9.14332 0.77448, 9....
  2  4  2  POLYGON ((3.35668 1.92552, 3.42703 2.20634, 3....,
  